In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from models.model import *
parameters = ParameterSettings()
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [40]:
# Sample agent
agent_trait_vec = torch.rand((parameters.n_agents, parameters.d_traits), device=parameters.device)
agent_belief_vec = torch.rand((parameters.n_agents, parameters.d_beliefs), device= parameters.device)

print(agent_trait_vec, agent_belief_vec)

tensor([[0.9775, 0.9335, 0.2647,  ..., 0.2080, 0.5915, 0.5374],
        [0.9216, 0.1446, 0.9384,  ..., 0.4736, 0.6774, 0.5878],
        [0.4946, 0.9110, 0.3946,  ..., 0.2123, 0.7718, 0.9994],
        ...,
        [0.1175, 0.4216, 0.9133,  ..., 0.7227, 0.3405, 0.4519],
        [0.5436, 0.5858, 0.1050,  ..., 0.1489, 0.8124, 0.4640],
        [0.2893, 0.3613, 0.7100,  ..., 0.1559, 0.2104, 0.2467]],
       device='cuda:0') tensor([[0.4751, 0.4561, 0.4560,  ..., 0.0499, 0.3307, 0.4459],
        [0.1199, 0.9788, 0.5746,  ..., 0.9798, 0.7724, 0.3735],
        [0.1806, 0.4023, 0.6209,  ..., 0.4793, 0.2839, 0.1070],
        ...,
        [0.3089, 0.7981, 0.0675,  ..., 0.9288, 0.7366, 0.4985],
        [0.1892, 0.1563, 0.5384,  ..., 0.9828, 0.1302, 0.3770],
        [0.8848, 0.4040, 0.0261,  ..., 0.2207, 0.4079, 0.3936]],
       device='cuda:0')


In [41]:
latents, weights = model.hypernet.forward(agent_trait_vec, agent_belief_vec)

In [42]:
obs_vec = torch.rand((parameters.n_agents, parameters.d_obs), device=parameters.device)
com_vec  = torch.rand((parameters.n_agents, parameters.d_comm_state), device=parameters.device)

In [43]:
p_weights, b_weights, e_weights, f_weights, d_weights, u_weights = weights
Q, h, ze = model.actor_encoder.forward(obs_vec, agent_belief_vec, com_vec, p_weights, b_weights, e_weights)

In [44]:
messages = []

for i in range(0, parameters.n_agents):
    Mij = torch.rand((parameters.n_agents, parameters.d_relation), device=parameters.device)
    weight = expand_weights(Mij.shape[0], i, f_weights)
    message = model.filter.forward(ze[i], Mij, weight)
    messages.append(message)

In [45]:
for i in range(0, parameters.n_agents):
    message = torch.tensor(messages[i][0], device = parameters.device).reshape(1, -1)
    Mij = torch.rand((1, parameters.d_relation), device=parameters.device)
    d_weight = expand_weights(1, i, d_weights)

    
    umw, umb, usw, usb = u_weights
    um_weights = umw, umb 
    us_weights = usw, usb 

    um_weight = expand_weights(1, i, um_weights)
    us_weight = expand_weights(1, i, us_weights)

    u_weight = *um_weight, *us_weight

    zdj , Mip = model.decoder_update.forward(message, Mij, d_weight, u_weight)



C:\Users\Joaquin\AppData\Local\Temp\ipykernel_1728\3507607844.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  message = torch.tensor(messages[i][0], device = parameters.device).reshape(1, -1)


# Parameter Counts

In [46]:
model.param_count()


        Hypernet: 7139936 
        Actor Encoder: 213888 
        Filter: 4736 
        Decoder: 133440 
        Total: 7492000
        
